# I. Bayes Estimator and Bayes Risk

We simulate an experiment to determine the likelihood of a person being a professional football player, an amateur, or neither, based on the position they usually play. 

## Question 1 (M)

#### A. Setup

##### Input Space \(X\)
The input space \(X = \{0, 1, 2, 3\}\) represents the positions played on a football field:
- \(0\) - Goalkeeper
- \(1\) - Defender
- \(2\) - Midfielder
- \(3\) - Attacker

##### Output Space \(Y\)
The output space \(Y = \{0, 1, 2\}\) categorizes the football playing status of a person:
- \(0\) - Regular person
- \(1\) - Amateur
- \(2\) - Professional

##### Probability Model for \(X\)
\(X\) follows a multinomial distribution with a single trial and probabilities \([0.1, 0.2, 0.3, 0.4]\) for each of the positions respectively. This suggests that in our sample, attackers are most frequent, followed by midfielders, defenders, and goalkeepers.

#### B. Conditional Distributions \(Y | X\)
- $Y = Mult(1, [0.7, 0.2, 0.1])$ if $X = 0$
- $Y = Mult(1, [0.8, 0.15, 0.05])$ if $X = 1$
- $Y = Mult(1, [0.9, 0.08, 0.02])$ if $X >= 2$


#### C. Bayes Estimator


- $P(Y=0) = \sum_{i=0}^3 P(Y=0|X=x_i)*P(X=x_i) = 0.86$
- $P(Y=1) = \sum_{i=0}^3 P(Y=1|X=x_i)*P(X=x_i) = 0.106$
- $P(Y=2) = \sum_{i=0}^3 P(Y=2|X=x_i)*P(X=x_i) = 0.0304$

$f^*(x) = 0$

#### D. Bayes Risk

$R(f^*) = E[I(Y, f^*(x))] = \sum_{i=0}^3 P(Y \neq 0 | X=x_i)*P(X=x_i) = 0.14$

## Question 2 (C)

- Bayes estimator : $f^*(x) = 0$
- Estimator f1 : $f_1(x) = 2$ if $X=0$ and $0$ otherwise

In [23]:
import numpy as np

def bayes() -> None:

    n_samples = int(1e6)

    X = np.random.multinomial(1, [0.1, 0.2, 0.3, 0.4], size=n_samples)

    # Copy X for parameters
    multinomial_parameters = np.zeros((n_samples, 3))  # Assuming there are 3 categories
    # Set parameters for each category using boolean indexing
    multinomial_parameters[X[:, 0] == 1] = [0.7, 0.2, 0.1]
    multinomial_parameters[X[:, 1] == 1] = [0.8, 0.15, 0.05]
    multinomial_parameters[(X[:, 2] == 1) | (X[:, 3] == 1)] = [0.9, 0.08, 0.02]

    # Generate new multinomial distribution based on updated parameters
    y = np.array([np.random.multinomial(1, p) for p in multinomial_parameters])

    y_pred_bayes = np.zeros((n_samples, 3))
    y_pred_bayes[:, 0] = 1

    # print(f"Bayes estimator: {y_pred_bayes}")
    y_pred_f1 = np.zeros((n_samples, 3))
    y_pred_f1[X[:, 0] == 1] = [0, 0, 1]
    y_pred_f1[(X[:, 1] == 1) | (X[:, 2] == 1) | (X[:, 3] == 1)] = [1, 0, 0]

    emperical_risk_bayes = np.mean(np.any(y != y_pred_bayes, axis=1))
    emperical_risk_f1 = np.mean(np.any(y != y_pred_f1, axis=1))

    print(f"Empirical risk for Bayes estimator: {emperical_risk_bayes}")
    print(f"Empirical risk for f1 estimator: {emperical_risk_f1}")

bayes()

Empirical risk for Bayes estimator: 0.140026
Empirical risk for f1 estimator: 0.199661


# II. Bayes risk with absolute loss

## Question 0 (M)

- A function that respects the requirements is $f(x) = x^3$
- $f'(x) = 3x^2$ and $f'(x_0) = 0 \iff x_0 = 0$
- $f''(x) = 6x$ and $f''(x_0) = 0 \iff x_0 = 0$

In this example $x_0$ is an inflection point, not a local extremum ($f''(x_0) = 0$).

## Question 1 (M + C)

#### A. Setup

##### Input Space \(X\)
Twe input space X is a continuous space, $ X \in \mathbb{R} $.
In our scenario, X is uniformly distributed over the interval [-5; 5].

##### Output Space \(Y\)
The output space Y is also a continuous space, $ Y \in \mathbb{R} $.
In our scenario, Y is determined by a linear relationship with X.

##### Probability Model for \(X\, Y\)

The relationship between X and  can be modeled as follows:

$Y = 2X + ϵ$

ϵ represents the noise and is uniformly distributed over the interval [0; 1].

Thus, $Y∣X∼Uniform(2X,2X+1)$

#### B. Predictors

$h(x) = 2x$

$f^*_{squared}(x) = 2x + 0.5$

#### C. Squared and Absolute Risks

$R_{squared(f^*)} = E[(Y - f^*(X))^2]$

$R_{absolute(f^*)} = E[|Y - f^*(X)|]$

In [32]:
import numpy as np
import matplotlib.pyplot as plt

def empirical_risk_absolute(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def empirical_risk_squared(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def absolute_squared_risk() -> None:
    np.random.seed(42)
    n_samples = int(1e6)

    X = np.random.uniform(-5, 5, size=n_samples)
    epsilon = np.random.uniform(0, 1, size=n_samples)
    Y = 2 * X + epsilon
    
    # Predictors
    f_squared = 2 * X + 0.5
    h = 2 * X

    # Compute empirical risks
    R_absolute_f_squared = empirical_risk_absolute(Y, f_squared)
    R_absolute_h = empirical_risk_absolute(Y, h)
    
    R_squared_f_squared = empirical_risk_squared(Y, f_squared)
    R_squared_h = empirical_risk_squared(Y, h)
    
    print("Empirical risks for f_squared (2x + 0.5):")
    print(f"  Absolute Loss: {R_absolute_f_squared}")
    print(f"  Squared Loss: {R_squared_f_squared}")
    
    print("\nEmpirical risks for h (2x):")
    print(f"  Absolute Loss: {R_absolute_h}")
    print(f"  Squared Loss: {R_squared_h}")

absolute_squared_risk()

Empirical risks for f_squared (2x + 0.5):
  Absolute Loss: 0.2497995290404182
  Squared Loss: 0.08324117421794368

Empirical risks for h (2x):
  Absolute Loss: 0.499478655298728
  Squared Loss: 0.3327198295166717


## Question 2 (M)

For a fixed x, we are searching $z \in \mathbb{R}$ that : $f^*_{squared}(x) = argmin(g(z))$.
In particular z must be the median of the conditional density of  $p_{Y|X=x}(y)$.

Indeed
- If $p_{Y|X=x}(y)$ is symmetric around its median, then $f^*_{absolute}(x)$ equals this median.
- Otherwiser, f is the point z such that the probability of being below z equals the probability of being above z.